In [2]:
# %%
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.storage.storage_context import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display

from llama_index.llms.openai import OpenAI
import chromadb

# %%



# import openai
# openai.api_key = "sk-ggyhlvddzgovjmjovtnpykqkywkfqbbfpllobangmpznsclj"

# openai.api_base = "https://api.siliconflow.cn/v1/chat/completions"
# llm = OpenAI(model='deepseek-chat',api_key=openai.api_key, base_url=openai.base_url)
# # llm = OpenAI(api_key=openai.api_key, base_url=openai.base_url)
# Settings.llm = OpenAI(model="deepseek-ai/DeepSeek-V3",
#                       api_key=openai.api_key,
#                       base_url=openai.base_url)



from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike

model_list =[
    'Qwen/Qwen2.5-7B-Instruct',
    'LoRA/Qwen/Qwen2.5-7B-Instruct',
    'Vendor-A/Qwen/Qwen2.5-72B-Instruct',
    'deepseek-ai/DeepSeek-R1-Distill-Llama-8B',
    'deepseek-ai/DeepSeek-R1'
    ]

Settings.llm = OpenAILike(
    model=model_list[2],  # 修改为正确的模型名称
    api_base="https://api.siliconflow.cn/v1",  # 修改 API 基础URL
    api_key="sk-ggyhlvddzgovjmjovtnpykqkywkfqbbfpllobangmpznsclj",
    is_chat_model=True
)

# %%
import os

jinaai_api_key = "jina_d1a2dcf77d894b6e82d185ed49316c1bOMnqubY9dJy0j21TXTgl216IR1VJ"
os.environ["JINAAI_API_KEY"] = jinaai_api_key

from llama_index.embeddings.jinaai import JinaEmbedding

text_embed_model = JinaEmbedding(
    api_key=jinaai_api_key,
    model="jina-embeddings-v3",
    # choose `retrieval.passage` to get passage embeddings
    task="retrieval.passage",
)

# %%
# 创建client和collection时使用get_or_create=True参数
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart", get_or_create=True)
# %%


# define embedding function
embed_model = text_embed_model

# load documents
documents = SimpleDirectoryReader("./data").load_data()

# save to disk

# 保存到磁盘时已经使用了get_or_create=True，这部分不需要修改
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

# Query Data from the persisted index
query_engine = index.as_query_engine(
    response_mode="compact",
    verbose=True  # 启用详细输出模式
)


In [6]:
# ... existing code ...

# 添加必要的导入
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision  # 替换 ContextRelevancy
)
from datasets import Dataset
import pandas as pd
import random
import json

In [10]:



def generate_test_cases(num_cases=100):
    """生成测试用例"""
    templates = [
        "详细解释{}攻击的工作原理是什么？",
        "如何防御{}攻击？",
        "{}攻击的危害有哪些？",
        "请用STIX2.0格式描述{}攻击的威胁情报",
        "分析{}攻击的特征指标(IOC)有哪些？"
    ]
    
    attack_types = [
        "LFA", "DDoS", "SQL注入", "XSS", "CSRF",
        "水坑攻击", "钓鱼攻击", "中间人攻击", "密码暴力破解",
        "反序列化", "文件包含", "命令注入", "目录遍历"
    ]
    
    test_cases = []
    for _ in range(num_cases):
        template = random.choice(templates)
        attack = random.choice(attack_types)
        question = template.format(attack)
        test_cases.append({"question": question})
    
    return test_cases
 # 1. 生成测试用例
test_cases = generate_test_cases(num_cases=100)
# 1.1 保存测试用例为json
with open('test_cases.json', 'w', encoding='utf-8') as f:
    json.dump(test_cases, f, ensure_ascii=False, indent=2)
print(f"已生成{len(test_cases)}个测试用例并保存到test_cases.json")


已生成100个测试用例并保存到test_cases.json


In [11]:
import json

def generate_and_save_answers(test_cases, query_engine, output_file="rag_responses.json"):
    """使用测试用例生成答案并保存结果"""
    results = []
    for i, test_case in enumerate(test_cases):
        try:
            response = query_engine.query(test_case["question"])
            result = {
                "question": test_case["question"],
                "contexts": [node.text for node in response.source_nodes],
                "answer": str(response),
                "ground_truths": [""]  # 如果有标准答案可以填充
            }
            results.append(result)
            
            # 每处理10个问题保存一次，防止中断丢失数据
            if (i + 1) % 10 == 0:
                with open(output_file, 'w', encoding='utf-8') as f:
                    json.dump(results, f, ensure_ascii=False, indent=2)
                print(f"已处理 {i + 1} 个问题")
                
        except Exception as e:
            print(f"处理问题时出错: {str(e)}")
            continue
    
    # 最后保存一次
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    return results

# 1.2 读取测试用例
with open('test_cases.json', 'r', encoding='utf-8') as f:
    test_cases = json.load(f)

# 2. 生成答案并保存
results = generate_and_save_answers(test_cases, query_engine)


已处理 10 个问题
已处理 20 个问题
已处理 30 个问题
已处理 40 个问题
已处理 50 个问题
已处理 60 个问题
已处理 70 个问题
已处理 80 个问题
已处理 90 个问题
已处理 100 个问题


In [ ]:
import os
import json
import time
# ... existing code ...

# 添加必要的导入
from ragas import evaluate, EvaluationDataset, SingleTurnSample
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision  # 替换 ContextRelevancy
)
from datasets import Dataset
import pandas as pd
import random
import json
from ragas.run_config import RunConfig

def evaluate_results(results_file="rag_responses.json", output_file="rag_evaluation_results.csv"):
    """评估生成的结果"""
    try:
        # 设置 OpenAI API 密钥
        os.environ["OPENAI_API_KEY"] = "sk-ggyhlvddzgovjmjovtnpykqkywkfqbbfpllobangmpznsclj"
        os.environ["OPENAI_API_BASE"] = "https://api.siliconflow.cn/v1"
        
        # 读取保存的结果
        with open(results_file, 'r', encoding='utf-8') as f:
            results = json.load(f)
        
        # 准备评估样本
        samples = []
        for result in results:
            sample = SingleTurnSample(
                user_input=result["question"],
                response=result["answer"],
                retrieved_contexts=result["contexts"],
                reference=result["ground_truths"][0] if result["ground_truths"] else ""
            )
            samples.append(sample)
            
        # 创建评估数据集
        eval_dataset = EvaluationDataset(samples=samples)
            
        # 配置评估用的LLM
        from ragas.llms import LlamaIndexLLMWrapper
        from llama_index.llms.openai_like import OpenAILike
        from openai import OpenAI
        evaluator_llm = LlamaIndexLLMWrapper(OpenAILike(
            api_base="https://api.siliconflow.cn/v1",  # 修改 API 基础URL
            api_key="sk-ggyhlvddzgovjmjovtnpykqkywkfqbbfpllobangmpznsclj",
            model="Vendor-A/Qwen/Qwen2.5-72B-Instruct"
        ))
        
        # 初始化评估指标
        from ragas.metrics import (
            Faithfulness,
            AnswerRelevancy, 
            ContextPrecision,
            ContextRecall,
        )
        
        metrics = [
            Faithfulness(llm=evaluator_llm),
            AnswerRelevancy(llm=evaluator_llm),
            ContextPrecision(llm=evaluator_llm),
            ContextRecall(llm=evaluator_llm),
        ]

        # 设置运行配置
        my_run_config = RunConfig(max_workers=1, timeout=120)
        
        # 执行评估
        evaluation_results = evaluate(
            dataset=eval_dataset,
            metrics=metrics,
            run_config=my_run_config
        )
            
        # 保存评估结果
        evaluation_results.to_csv(output_file, index=False)
        
        return evaluation_results
        
    except Exception as e:
        print(f"评估过程中出现错误: {str(e)}")
        import traceback
        traceback.print_exc()  # 打印完整的错误堆栈
        return None

# 执行评估
evaluation_results = evaluate_results()

if evaluation_results is not None:
    print("\n性能评估结果:")
    print(evaluation_results)